In [4]:
import os

print("Current directory before change:", os.getcwd())

try:
    
    os.chdir("../")
    print("Current directory after change:", os.getcwd())

except FileNotFoundError:
    print("""
        FileNotFoundError - The specified directory does not exist or you are already in the root.
        If the code already worked once, do not run it again.
    """)

Current directory before change: /home/jacobo/workshop_02/notebooks
Current directory after change: /home/jacobo/workshop_02


In [1]:
import time
import requests

# 1) URL base de la API v2
BASE_URL = "https://musicbrainz.org/ws/2"

# 2) Tu token de acceso (opcional para lecturas públicas)
TOKEN = "jDVD3Je9aRhX2ugltZxL7gROvpvglClltKIqq27x"

# 3) User-Agent obligatorio para MusicBrainz y Authorization si fuese necesario
HEADERS = {
    "User-Agent": "MiProyecto/1.0 (tu_email@ejemplo.com)",
    "Authorization": f"Bearer {TOKEN}"
}

def detalles_artista(mbid):
    """
    Recupera datos detallados de un artista por su MBID
    """
    endpoint = f"{BASE_URL}/artist/{mbid}"
    params = {
        "fmt": "json",
        # inc: campos extra a incluir: aliases, tags, ratings y release-groups
        "inc": "aliases+tags+ratings+release-groups"
    }

    resp = requests.get(endpoint, headers=HEADERS, params=params)
    resp.raise_for_status()
    return resp.json()

if __name__ == "__main__":
    # Ejemplo de MBID de Coldplay
    mbid = "cc197bad-dc9c-440d-a5b5-d52ba2e14234"
    
    # Llamada a la API
    info = detalles_artista(mbid)
    
    # Mostrar algunos campos
    print("Nombre:", info["name"])
    print("País:", info.get("country"))
    print("Número de grupos de lanzamiento:", len(info.get("release-groups", [])))
    
    # Para respetar el rate-limit de 1 petición/segundo (si haces múltiples peticiones en un bucle)
    time.sleep(1)


Nombre: Coldplay
País: GB
Número de grupos de lanzamiento: 25


In [8]:
import time
import requests
import pandas as pd

# Configuración de la API
BASE_URL = "https://musicbrainz.org/ws/2"
HEADERS = {
    "User-Agent": "MiProyecto/1.0 (tu_email@ejemplo.com)"
}

def buscar_mbid_artista(nombre):
    """
    Busca el MBID del primer artista cuyo nombre coincida con 'nombre'
    """
    url = f"{BASE_URL}/artist/"
    params = {
        "query": nombre,
        "fmt": "json",
        "limit": 1
    }
    resp = requests.get(url, headers=HEADERS, params=params)
    resp.raise_for_status()
    artists = resp.json().get("artists", [])
    return artists[0]["id"] if artists else None

def detalles_artista(mbid):
    """
    Recupera datos detallados de un artista por su MBID
    y devuelve un diccionario con campos seguros.
    """
    url = f"{BASE_URL}/artist/{mbid}"
    params = {
        "fmt": "json",
        # Sólo sub-recursos válidos:
        "inc": "aliases+tags+ratings+release-groups"
    }
    resp = requests.get(url, headers=HEADERS, params=params)
    resp.raise_for_status()
    art = resp.json()

    # Extraer campos con comprobación de None
    tags = art.get("tags") or []
    rating = art.get("rating") or {}
    release_groups = art.get("release-groups") or []
    area = art.get("area")
    life = art.get("life-span")

    # Primer año de lanzamiento
    years = []
    for rg in release_groups:
        date = rg.get("first-release-date")
        if date:
            try:
                years.append(int(date.split("-")[0]))
            except ValueError:
                pass

    return {
        "tags": ";".join(t.get("name", "") for t in tags),
        "rating": rating.get("value"),
        "first_year": min(years) if years else None,
        "area": area.get("name") if area else None,
        "active_begin": life.get("begin") if life else None,
        "active_end": life.get("end") if life else None
    }

def enriquecer_artistas_unicos(path_in, path_out):
    # 1) Carga tu dataset
    df = pd.read_csv(path_in)

    # 2) Obtén artistas únicos y normaliza nombres (solo antes de ';')
    artistas_raw = df["artists"].dropna().unique()
    artistas_map = { raw: raw.split(";")[0].strip() for raw in artistas_raw }

    # 3) Prepara diccionario para resultados
    resultados = {
        "artists": [],
        "artist_mbid": [],
        "artist_tags": [],
        "artist_rating": [],
        "artist_first_year": [],
        "artist_area": [],
        "artist_active_begin": [],
        "artist_active_end": []
    }

    # 4) Itera solo artistas únicos
    for raw, nombre in artistas_map.items():
        datos = {
            "artists": raw,
            "artist_mbid": None,
            "artist_tags": None,
            "artist_rating": None,
            "artist_first_year": None,
            "artist_area": None,
            "artist_active_begin": None,
            "artist_active_end": None
        }
        try:
            # a) Buscar MBID
            mbid = buscar_mbid_artista(nombre)
            datos["artist_mbid"] = mbid
            time.sleep(1)  # rate‑limit: 1 petición/segundo

            if mbid:
                # b) Obtener detalles enriquecidos
                info = detalles_artista(mbid)
                datos.update({
                    "artist_tags": info["tags"],
                    "artist_rating": info["rating"],
                    "artist_first_year": info["first_year"],
                    "artist_area": info["area"],
                    "artist_active_begin": info["active_begin"],
                    "artist_active_end": info["active_end"]
                })
                time.sleep(1)  # rate‑limit

        except Exception as e:
            print(f"[Error con '{raw}']: {e}")

        # Guarda en el diccionario
        for k, v in datos.items():
            resultados[k].append(v)

    # 5) Construye DataFrame de artistas enriquecidos
    df_art = pd.DataFrame(resultados)

    # 6) Merge con el DataFrame original
    df_final = df.merge(df_art, on="artists", how="left")

    # 7) Guarda el resultado
    df_final.to_csv(path_out, index=False)
    print(f"Dataset enriquecido guardado en {path_out}")

if __name__ == "__main__":
    enriquecer_artistas_unicos(
        path_in="data/dataset_spotify.csv",
        path_out="data/dataset_spotify_enriched.csv"
    )


KeyboardInterrupt: 